## How to use ccall properly

This notebook demonstrates a couple of ways to make calls to C functions that 
accept either C structs or pointer to C struct.

Cases as follows:

- test1: takes no argument
- test2: takes a struct argument
- test3: takes a pointer to a struct argument
- test4: takes a pointer to a struct argument and returns a pointer to a struct

In [4]:
const output = "/tmp/test.dylib"

"/tmp/test.dylib"

In [5]:
cprog = """
#include <stdlib.h>

typedef struct 
{
    int a;
    double b;
} FOO, *FOOP;

FOO test1() {
    FOO x;
    x.a = 1;
    x.b = 2.0;
    return x;
}

FOO test2(FOO x) {
    return x;
}

FOO test3(FOOP xp) {
    FOO y;
    y.a = xp->a;
    y.b = xp->b;
    return y;
}

FOOP test4(FOOP xp) {
    FOOP yp = (FOOP) malloc(sizeof(FOO));
    yp->a = xp->a;
    yp->b = xp-> b;
    return yp;
}
""";

In [6]:
open(`gcc -fPIC -O3 -xc -shared -o $output -`, "w") do f
     print(f, cprog)
end

In [7]:
struct FOO
    a::Cint
    b::Cdouble
end

### Test1 - No argument & returns statically constructed struct

In [8]:
x = ccall((:test1, output), FOO, ())

FOO(1, 2.0)

In [9]:
GC.gc(true)
x

FOO(1, 2.0)

### Test2 - Struct argument & returns the argument itself

In [11]:
y = ccall((:test2, output), FOO, (FOO,), x)

FOO(1, 2.0)

In [13]:
# same object?
x === y

true

### Test3 - Pointer-to-Struct argument & returns static Struct

In [19]:
x

FOO(1, 2.0)

In [23]:
@show p = ccall((:test3, output), FOO, (Ptr{FOO},), Ref(x))
@show a = ccall((:test3, output), FOO, (Ptr{FOO},), Ref(x))
;

p = ccall((:test3, output), FOO, (Ptr{FOO},), Ref(x)) = FOO(1, 2.0)
a = ccall((:test3, output), FOO, (Ptr{FOO},), Ref(x)) = FOO(1, 2.0)


In [26]:
# same?  that's interesting...
p === a

true

### Test4 - Pointer-to-Struct argument & returns malloc'ed Pointer-to-Struct

In [27]:
# use Ref to pass a proper address to the C function
q = ccall((:test4, output), Ptr{FOO}, (Ptr{FOO},), Ref(x))

Ptr{FOO} @0x00007fc8667b9b70

In [28]:
# Reconstruct the julia object
q2 = unsafe_load(q)

FOO(1, 2.0)

In [32]:
# nothing special, just defining a function here
bar(value) = ccall((:test4, output), Ptr{FOO}, (Ptr{FOO},), Ref(value))
x2 = bar(x)

Ptr{FOO} @0x00007fc8666ac720

In [33]:
# can still convert it back to julia object
unsafe_load(x2)

FOO(1, 2.0)

In [35]:
# what is the type of Ref(x) anyways?
typeof(Ref(x))

Base.RefValue{FOO}

In [36]:
# so we can define a function as such.
woz(xr::Base.RefValue{FOO}) = ccall((:test4, output), Ptr{FOO}, (Ptr{FOO},), xr)

woz (generic function with 1 method)

In [37]:
woz(Ref(x))

Ptr{FOO} @0x00007fc86638d0c0

In [39]:
woz(Ref(x))

Ptr{FOO} @0x00007fc8660d2e80

In [40]:
unsafe_load(woz(Ref(x)))

FOO(1, 2.0)